In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3/transformers/8b-hf/1/model.safetensors.index.json
/kaggle/input/llama-3/transformers/8b-hf/1/model-00003-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-hf/1/config.json
/kaggle/input/llama-3/transformers/8b-hf/1/LICENSE
/kaggle/input/llama-3/transformers/8b-hf/1/model-00001-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-hf/1/USE_POLICY.md
/kaggle/input/llama-3/transformers/8b-hf/1/tokenizer.json
/kaggle/input/llama-3/transformers/8b-hf/1/tokenizer_config.json
/kaggle/input/llama-3/transformers/8b-hf/1/example_text_completion.py
/kaggle/input/llama-3/transformers/8b-hf/1/requirements.txt
/kaggle/input/llama-3/transformers/8b-hf/1/model-00004-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-hf/1/eval_details.md
/kaggle/input/llama-3/transformers/8b-hf/1/special_tokens_map.json
/kaggle/input/llama-3/transformers/8b-hf/1/model-00002-of-00004.safetensors
/kaggle/input/llama-3/transformers/8b-hf/1/example_chat_completion.py
/kaggle/inpu

In [2]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.5 MB/s eta 0:00:0000:0100:01


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_name = "/kaggle/input/llama-3/transformers/8b-hf/1/"

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quanty_type = "fp4", 
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quanty = True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    torch_dtype = torch.float16,
    device_map = "auto",
    trust_remote_code = True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Unused kwargs: ['bnb_4bit_quanty_type', 'bnb_4bit_use_double_quanty']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
def generate_prompts(q_dict, context=None, context_type=None):
    context_str = ''
    if context is not None:
        # Context: "Democrat", "Republican", "Independent"
        # Context Type: "QA", "BIO", "PORTRAY"
        if context_type == 'QA':
            context_options = {'republican': '1', 'democrat': '2', 'independent':'3'}
            option = context_options.get(context.strip().lower(), '4')
            context_str = f"""
Question: In politics today, do you consider yourself a
1. Republican
2. Democrat
3. Independent
4. Something else
5. Refused
Answer: {option}
"""
        elif context_type == 'BIO':
            context_str = f"""
Below you will be asked to provide a short description of your political affiliation and then answer some questions.
Description: In politics today, I consider myself a {context.strip().title()}.
"""
        
        elif context_type == 'PORTRAY':
            context_str = f"""
Answer the following question as if in politics today, you considered yourself a {context.strip().title()}.
"""
    
    main_question = q_dict['main_question']
    sub_questions = q_dict['sub_questions']
    
    expected_answers = [str(ans["id"]) for ans in q_dict["answers"]]
    if not sub_questions:
        prompt = f"""
{context_str}

Question: {main_question}
"""
        
        for ans in q_dict['answers']:
            prompt += f"{ans['id']}: {ans['text']}\n"
        prompt += f"Answer: "
        return [prompt], expected_answers
    
    prompts = []
    for sq in sub_questions:
        prompt = f"""
{context_str}

Question: {main_question}
{sq['id']}: {sq['text']}
"""
        for ans in q_dict['answers']:
            prompt += f"{ans['id']}: {ans['text']}\n"
        prompt += f"Answer: "
        prompts.append(prompt)
    
    return prompts, expected_answers

In [5]:
def get_distribution(prompt, expected_answers) -> dict:
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits[:, -1, :]  # Get logits for the last token

    # Calculate probabilities and log probabilities
    probabilities = torch.softmax(logits, dim=-1)
    log_probs = torch.log(probabilities)

    # Get top k candidates
    top_k = 100 
    top_probabilities, top_indices = torch.topk(log_probs, top_k)

    # Decode the top tokens to words
    top_words = tokenizer.convert_ids_to_tokens(top_indices[0].tolist())

    # Display results
    probabilities = {}
    for word, prob in zip(top_words, top_probabilities[0].tolist()):
        # print(f"Word: {word}, Log Probability: {prob:.4f}")
        probabilities[word] = prob
    
    distribution = {}
    for ans in expected_answers:
        if ans in probabilities:
            distribution[ans] = probabilities[ans]
    
    # distribution = {k:probabilities[k] for k in expected_answers}
    
    return distribution
    

In [6]:
import json

In [7]:
for i in range(1, 20):
    foo = open(f'/kaggle/working/output_{i}.txt', 'w')
    with open(f'/kaggle/input/pewdata/{i}.json') as f:
        q = json.load(f)
    prompts, expected_answers = generate_prompts(q)
    for prompt in prompts:
        print(prompt, file=foo)
        dist = get_distribution(prompt, expected_answers)
        print(dist, file=foo)
        print('===', file=foo)
    foo.close()

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
